In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Read and process the data

# Split into train and test dataset

# Build recommender system

# Make prediction and calcute RMSE error